In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [5]:
#pip install -r requirements.txt

In [6]:
import torch

In [7]:
torch.cuda.is_available()

True

In [61]:
from data_util import get_data_from_kaggle,sentiment_analysis , aggregate_tweet, restructure_date_information, get_static_df, one_label_scale_static_df, scale_stock_data
from constant import Constant
import pandas as pd
from TFT import TFT_embedding, TFT

In [9]:
TFT

TFT.tft.TFT

In [10]:
constant = Constant()
constant

In [11]:
stock, tweet = get_data_from_kaggle()

In [12]:
tweet = pd.read_csv('processed.csv')
#tweet

In [13]:
aggregated_tweet = aggregate_tweet(tweet)
aggregated_tweet

,Stock Name,Date,Positive_Avg,Neutral_Avg,Negative_Avg,Positive_Count,Neutral_Count,Negative_Count
0,AAPL,2021-09-30,0.322527,0.465190,0.212283,3,4,0
1,AAPL,2021-10-01,0.063409,0.574801,0.361790,0,8,3
2,AAPL,2021-10-02,0.053914,0.569441,0.376645,0,2,2
3,AAPL,2021-10-03,0.365725,0.546062,0.088213,0,1,0
4,AAPL,2021-10-04,0.082445,0.740438,0.177116,0,10,0
...,...,...,...,...,...,...,...,...
5905,ZS,2022-09-18,0.002347,0.195517,0.802136,0,0,1
5906,ZS,2022-09-21,0.025520,0.272998,0.701482,0,0,1
5907,ZS,2022-09-27,0.060069,0.497830,0.442101,0,1,0
5908,ZS,2022-09-28,0.129386,0.716569,0.154045,0,1,0


In [14]:
stock['Date'] = pd.to_datetime(stock['Date'] ).dt.date

In [15]:
merged_df = pd.merge(aggregated_tweet, stock, on=['Date', 'Stock Name'], how='right')
merged_df = merged_df.fillna(0)

In [16]:
processed_df = restructure_date_information(merged_df)
#processed_df

In [17]:
static = get_static_df(processed_df,  constant.static_variables)

c:\Users\Luan\Documents\DL\TFT\data_util.py:74: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  static_df.fillna(0, inplace=True)


In [18]:
static[["sector", "country", "beta"]].loc["TSLA"].values

array(['Consumer Cyclical', 'United States', 2.295], dtype=object)

In [19]:
static_df = one_label_scale_static_df(static,constant.static_variables)

In [20]:
from data import TFT_Dataset
from torch.utils.data import DataLoader
x = TFT_Dataset(stock_df= processed_df , 
                static_df=static_df,
                constant_variable=constant, 
                history_length= 5, 
                prediction_length= 2)

In [21]:
loader = DataLoader(x, batch_size= 10)

In [22]:
count =0
for static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input, prediction in loader:
    break
static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input, prediction

(tensor([[0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818],
         [0.3658, 1.0000, 0.2182, 0.1214, 0.0701, 0.1094, 0.5818]]),
 tensor([[4, 5, 2],
         [4, 5, 2],
         [4, 5, 2],
         [4, 5, 2],
         [4, 5, 2],
         [4, 5, 2],
         [4, 5, 2],
         [4, 5, 2],
         [4, 5, 2],
         [4, 5, 2]], dtype=torch.int32),
 tensor([[[3.2253e-01, 4.6519e-01, 2.1228e-01, 3.0000e+00, 4.0000e+00,
           0.0000e+00, 1.4366e+02, 1

In [23]:
from data_util import get_feature_length

history_cat_feature_num_list, history_cont_feature_num = get_feature_length(processed_df, constant.feature_variables)
static_cat_feature_num_list , static_cont_feature_num  = get_feature_length(static, constant.static_variables)
future_feature_num          , _                        = get_feature_length(processed_df, constant.future_feature)
_                           , prediction_con           = get_feature_length(processed_df, constant.prediction_feature)

In [24]:
history_cat_feature_num_list

[12, 31, 5, 52]

In [27]:
static_input, history_input, future_emb = x

In [28]:
static_input.size()

torch.Size([5, 64, 4])

In [29]:
static_input.size()

torch.Size([5, 64, 4])

In [30]:
history_input.size()

torch.Size([5, 5, 64, 5])

In [31]:
future_emb.size()

torch.Size([5, 2, 64, 4])

In [32]:
len(future_emb)

5

In [33]:
feature_size = history_input.size(-1)

In [34]:
from TFT import GRN

In [35]:
feature_gru = [  GRN(64,64)     for i in range(feature_size)    ]

In [36]:
x = [ feature_gru[i](history_input[:,:,:,i]) for i in range(feature_size)    ]  

In [37]:
static_input

tensor([[[ 0.4002, -0.8560,  0.5732,  1.1735],
         [-0.1852, -0.3178,  1.0571, -0.0367],
         [-0.2055, -0.2240, -0.8033, -0.8358],
         ...,
         [ 0.0421, -0.2011,  1.2316,  0.8858],
         [ 0.2406, -1.3437,  0.1829,  0.9423],
         [ 0.3222,  1.3329, -0.0996,  0.3477]],

        [[ 0.4002, -0.8560,  0.5732,  1.1735],
         [-0.1852, -0.3178,  1.0571, -0.0367],
         [-0.2055, -0.2240, -0.8033, -0.8358],
         ...,
         [ 0.0421, -0.2011,  1.2316,  0.8858],
         [ 0.2406, -1.3437,  0.1829,  0.9423],
         [ 0.3222,  1.3329, -0.0996,  0.3477]],

        [[ 0.4002, -0.8560,  0.5732,  1.1735],
         [-0.1852, -0.3178,  1.0571, -0.0367],
         [-0.2055, -0.2240, -0.8033, -0.8358],
         ...,
         [ 0.0421, -0.2011,  1.2316,  0.8858],
         [ 0.2406, -1.3437,  0.1829,  0.9423],
         [ 0.3222,  1.3329, -0.0996,  0.3477]],

        [[ 0.4002, -0.8560,  0.5732,  1.1735],
         [-0.1852, -0.3178,  1.0571, -0.0367],
         [-0

In [38]:
history_input.size()

torch.Size([5, 5, 64, 5])

In [39]:
x[0]

tensor([[[ 0.1139, -1.1631, -0.4052,  ..., -1.0725, -0.5947, -0.7030],
         [ 0.1139, -1.1631, -0.4052,  ..., -1.0726, -0.5947, -0.7030],
         [ 0.1139, -1.1631, -0.4053,  ..., -1.0727, -0.5946, -0.7031],
         [ 0.1139, -1.1631, -0.4052,  ..., -1.0726, -0.5947, -0.7030],
         [ 0.1140, -1.1631, -0.4052,  ..., -1.0725, -0.5947, -0.7030]],

        [[ 0.1139, -1.1631, -0.4052,  ..., -1.0726, -0.5947, -0.7030],
         [ 0.1139, -1.1631, -0.4053,  ..., -1.0727, -0.5946, -0.7031],
         [ 0.1139, -1.1631, -0.4052,  ..., -1.0726, -0.5947, -0.7030],
         [ 0.1140, -1.1631, -0.4052,  ..., -1.0725, -0.5947, -0.7030],
         [ 0.1140, -1.1631, -0.4052,  ..., -1.0725, -0.5947, -0.7030]],

        [[ 0.1139, -1.1631, -0.4053,  ..., -1.0727, -0.5946, -0.7031],
         [ 0.1139, -1.1631, -0.4052,  ..., -1.0726, -0.5947, -0.7030],
         [ 0.1140, -1.1631, -0.4052,  ..., -1.0725, -0.5947, -0.7030],
         [ 0.1140, -1.1631, -0.4052,  ..., -1.0725, -0.5947, -0.7030],
  

In [40]:
from TFT import VariationSelection

In [41]:
static_input.size()

torch.Size([5, 64, 4])

In [42]:
v = VariationSelection(64,4)

In [43]:
y = v.forward(static_input)

In [44]:
torch.nn.functional.softmax(y,dim =-1).size()

torch.Size([5, 64])

In [45]:
y.size()

torch.Size([5, 64])

In [46]:
y.size()

torch.Size([5, 64])

In [47]:
f = VariationSelection(64,5,context_size = 64)

In [48]:
d = f.forward(history_input,y)

In [ ]:
x.size()

torch.Size([5, 5, 64, 1])

In [ ]:
d.size()


torch.Size([5, 5, 64])

In [176]:
emb = TFT(static_cat_feature_num_list= static_cat_feature_num_list,
             static_cont_feature_num=static_cont_feature_num,
             history_cat_feature_num_list= history_cat_feature_num_list,
             history_cont_feature_num=history_cont_feature_num,
             future_feature_list=future_feature_num,
             seq_len = 7,
             future_len = 2,

             hidden_size=64)
emb

TFT(
  (tft_embed): TFT_embedding(
    (static_cont): Linear(in_features=7, out_features=64, bias=True)
    (static_cat): ModuleList(
      (0): Embedding(16, 64)
      (1): Embedding(6, 64)
      (2): Embedding(3, 64)
    )
    (history_cont): Linear(in_features=11, out_features=64, bias=True)
    (history_cat): ModuleList(
      (0): Embedding(12, 64)
      (1): Embedding(31, 64)
      (2): Embedding(5, 64)
      (3): Embedding(52, 64)
    )
    (future_feature): ModuleList(
      (0): Embedding(12, 64)
      (1): Embedding(31, 64)
      (2): Embedding(5, 64)
      (3): Embedding(52, 64)
    )
  )
  (cs): VariationSelection(
    (group_GRN): GRN(
      (linear1): Linear(in_features=256, out_features=256, bias=True)
      (linear2): Linear(in_features=256, out_features=64, bias=True)
      (GLU): GLU(
        (linear4): Linear(in_features=64, out_features=256, bias=True)
        (linear5): Linear(in_features=64, out_features=256, bias=True)
      )
      (droput): Dropout(p=0.0, inpla

In [177]:
for static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input, prediction in loader:
    x = emb(static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input)
    break
print(x)

torch.Size([10, 64])
torch.Size([10, 64])
torch.Size([10, 64])
torch.Size([10, 64])
torch.Size([10, 5, 64])
torch.Size([10, 2, 64])
history_input torch.Size([10, 5, 64, 5])
x torch.Size([10, 5, 64])
residual torch.Size([10, 5, 64])
gated torch.Size([10, 5, 64])
x torch.Size([10, 2, 64])
residual torch.Size([10, 2, 64])
gated torch.Size([10, 2, 64])
static_enriched torch.Size([10, 7, 64])
attention_QKV torch.Size([10, 7, 7])
x torch.Size([10, 7, 7])
residual torch.Size([10, 7, 64])
gated torch.Size([10, 7, 64])
lstm torch.Size([10, 2, 64])
tensor([[[-0.5423,  0.8767, -0.3074],
         [-0.9336,  0.2998, -0.0934]],

        [[-1.1175,  0.1633,  0.0638],
         [-1.1000,  0.5756,  0.6080]],

        [[-0.9320,  0.5379,  0.3814],
         [-0.2832,  1.0623,  0.0298]],

        [[-0.4654,  1.3879,  0.1780],
         [-1.3852,  1.5010,  0.5490]],

        [[-1.1674,  1.3211,  0.9434],
         [-1.0305,  1.5124,  0.3802]],

        [[-1.3750,  1.6614,  0.1139],
         [-0.7757,  0.5982,

In [173]:
x

tensor([[[ 0.0687,  0.6977,  0.3785,  ...,  1.1539, -1.4019,  0.4181],
         [ 1.7472, -0.7559,  1.9793,  ...,  1.4521, -1.0171, -1.8192],
         [ 0.8419, -0.2184,  1.2573,  ..., -0.0700, -1.2749, -0.2932],
         ...,
         [ 1.2444, -0.4137,  1.0899,  ...,  1.6794, -1.9001, -1.0663],
         [ 0.7266,  0.3071, -1.0389,  ...,  1.3872,  1.5878,  2.1856],
         [ 0.7088,  1.9889,  0.0465,  ...,  1.2517,  2.2395,  0.9451]],

        [[ 1.7467, -0.7221,  1.9574,  ...,  1.4453, -1.0070, -1.8417],
         [ 0.8405, -0.1982,  1.2438,  ..., -0.0854, -1.2682, -0.3036],
         [ 1.5748, -0.7587,  0.8706,  ...,  0.3736, -2.3626, -0.7074],
         ...,
         [ 1.7713,  0.1461,  0.6986,  ...,  0.3021, -1.6423, -0.8768],
         [ 0.7476,  1.9508,  0.1232,  ...,  1.1785,  2.2025,  0.6015],
         [-1.3652,  1.1453, -1.2450,  ...,  1.6384, -0.2929, -0.4110]],

        [[ 0.8422, -0.2025,  1.2403,  ..., -0.0779, -1.2704, -0.3420],
         [ 1.5826, -0.7608,  0.8730,  ...,  0

In [136]:
 torch.triu(torch.full((5, 5), float('-inf')), 1).unsqueeze(0).size()


torch.Size([1, 5, 5])